In [1]:
%load_ext watermark
%watermark

%load_ext autoreload
%autoreload 2


# import standard libs
from IPython.display import display
from IPython.core.debugger import set_trace as bp
from pathlib import PurePath, Path
import sys
import time
from collections import OrderedDict as od
import re
import os
import json
import datetime
import pickle


# import python scientific stack
import pandas as pd
import pandas_datareader.data as web
pd.set_option('display.max_rows', 100)
from dask import dataframe as dd
from dask.diagnostics import ProgressBar
from multiprocessing import cpu_count
pbar = ProgressBar()
pbar.register()
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from numba import jit
import math
# import ffn


# import visual tools
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
import seaborn as sns

plt.style.use('seaborn-talk')
plt.style.use('bmh')
#plt.rcParams['font.family'] = 'DejaVu Sans Mono'
plt.rcParams['font.size'] = 9.5
plt.rcParams['font.weight'] = 'medium'
plt.rcParams['figure.figsize'] = 10,7
blue, green, red, purple, gold, teal = sns.color_palette('colorblind', 6)

RANDOM_STATE = 777

print()
%watermark -p pandas,pandas_datareader,dask,numpy,sklearn,statsmodels,scipy,ffn,matplotlib,seaborn

Last updated: 2024-09-18T12:34:30.117873-04:00

Python implementation: CPython
Python version       : 3.8.19
IPython version      : 8.12.2

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 23.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit


pandas           : 2.0.3
pandas_datareader: 0.10.0
dask             : 2023.5.0
numpy            : 1.24.4
sklearn          : 1.3.2
statsmodels      : 0.14.1
scipy            : 1.10.1
ffn              : not installed
matplotlib       : 3.7.3
seaborn          : 0.13.2



/var/folders/j0/8v9qcjfx15g5ftmsy5n0qrq80000gn/T/ipykernel_39206/178535154.py:46: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-talk')


In [2]:
import os

# Run the setup script
%run ../../config/setup_project.py

# Call the function to set up the project path
setup_project_path()

# Now you can import your modules
from src.utils import helper as h_
import src.ch_02.code_ch_02 as f_ch2
import src.ch_03.code_ch_03 as f_ch3

Project root added to sys.path: /Users/paulkelendji/Desktop/GitHub_paul/ML-Asset_Management
Config path added to sys.path: /Users/paulkelendji/Desktop/GitHub_paul/ML-Asset_Management/config
Current sys.path: ['/Users/paulkelendji/miniconda3/envs/financial_math/lib/python38.zip', '/Users/paulkelendji/miniconda3/envs/financial_math/lib/python3.8', '/Users/paulkelendji/miniconda3/envs/financial_math/lib/python3.8/lib-dynload', '', '/Users/paulkelendji/miniconda3/envs/financial_math/lib/python3.8/site-packages', '/Users/paulkelendji/miniconda3/envs/financial_math/lib/python3.8/site-packages/setuptools/_vendor', '/Users/paulkelendji/Desktop/GitHub_paul/ML-Asset_Management', '/Users/paulkelendji/Desktop/GitHub_paul/ML-Asset_Management/config']
Project root added to sys.path: /Users/paulkelendji/Desktop/GitHub_paul/ML-Asset_Management
Config path added to sys.path: /Users/paulkelendji/Desktop/GitHub_paul/ML-Asset_Management/config
Current sys.path: ['/Users/paulkelendji/miniconda3/envs/financ

---

# Exercises

In [3]:
# load ../data/variables_ch2.pkl
%run ../ch_02/code_ch_02.py

path = '../../data/variables_ch2.pkl'
import pickle
with open(path, 'rb') as f:
    bars = pickle.load(f)
    bar_time = pickle.load(f)
    
# df as bars['Dollar'].df_OLHC without 'cusum' column
df = bars['Dollar'].df_OLHC.drop(columns=['cusum'])
# For the purpose of this example, remove rows where time_close is duplicated
# (keep the first row)
# Remove rows where time_close is duplicated, keeping the first occurrence
df = df.drop_duplicates(subset='time_close', keep='first')
# set index as 'time_close'
df = df.set_index('time_close')
df

,time_open,open,low,high,close,vwap,volume
time_close,,,,,,,
2009-09-29 13:23:42,2009-09-28 09:30:00,50.7900,50.710,51.9600,51.730,51.430024,701459
2009-10-01 13:44:06,2009-09-29 13:26:31,51.7275,50.070,51.7600,50.270,50.908633,709392
2009-10-02 14:37:25,2009-10-01 13:45:10,50.2376,49.190,50.3166,49.761,49.836850,724111
2009-10-07 11:39:00,2009-10-02 14:38:03,49.7635,49.440,51.4700,51.090,50.606475,713127
2009-10-12 14:24:38,2009-10-07 11:40:15,51.0880,50.980,52.2252,51.990,51.675670,698754
...,...,...,...,...,...,...,...
2024-07-25 11:50:36,2024-07-24 15:51:05,186.6706,186.460,188.5800,188.400,187.387274,186271
2024-07-25 14:46:14,2024-07-25 11:50:40,188.4000,187.685,189.1286,187.685,188.380972,194394
2024-07-26 10:11:15,2024-07-25 14:46:35,187.6150,186.980,189.2300,189.020,187.749127,195797


In [4]:
"""
**4.1** In Chapter 3, we denoted as `t1` a pandas series of timestamps where the first barrier was touched, and the index was the timestamp of the observation. This was the output of the `getEvents` function.

- **(a)** Compute a `t1` series on dollar bars derived from E-mini S&P 500 futures tick data.
  
- **(b)** Apply the function `mpNumCoEvents` to compute the number of overlapping outcomes at each point in time.
  
- **(c)** Plot the time series of the number of concurrent labels on the primary axis, and the time series of exponentially weighted moving standard deviation of returns on the secondary axis.
  
- **(d)** Produce a scatterplot of the number of concurrent labels (x-axis) and the exponentially weighted moving standard deviation of returns (y-axis). Can you appreciate a relationship?
"""

'\n**4.1** In Chapter 3, we denoted as `t1` a pandas series of timestamps where the first barrier was touched, and the index was the timestamp of the observation. This was the output of the `getEvents` function.\n\n- **(a)** Compute a `t1` series on dollar bars derived from E-mini S&P 500 futures tick data.\n  \n- **(b)** Apply the function `mpNumCoEvents` to compute the number of overlapping outcomes at each point in time.\n  \n- **(c)** Plot the time series of the number of concurrent labels on the primary axis, and the time series of exponentially weighted moving standard deviation of returns on the secondary axis.\n  \n- **(d)** Produce a scatterplot of the number of concurrent labels (x-axis) and the exponentially weighted moving standard deviation of returns (y-axis). Can you appreciate a relationship?\n'

### **4.1** In Chapter 3, we denoted as `t1` a pandas series of timestamps where the first barrier was touched, and the index was the timestamp of the observation. This was the output of the `getEvents` function.

**(a)** Compute a `t1` series on dollar bars derived from E-mini S&P 500 futures tick data.

In [5]:
# Step 1 : get the daily volatility
close = df['close']
dailyVol = f_ch3.getDailyVol(close, span0=100).dropna()

# Step 2 : Detect events (significant points in time where price changes)
yt = close.pct_change().dropna()
tEvents = f_ch3.getTEvents(yt, h=dailyVol)

# Step 3 : Obtain t1 series
NUM_DAYS = 10
t1 = f_ch3.addVerticalBarrier(tEvents, close, numDays=NUM_DAYS)
t1


2009-10-02 14:37:25   2009-10-15 12:03:22
2009-10-07 11:39:00   2009-10-20 11:12:37
2009-10-20 11:12:37   2009-11-02 10:30:21
2009-10-27 11:06:15   2009-11-10 10:55:00
2009-11-02 10:30:21   2009-11-13 12:12:50
                              ...        
2024-07-02 14:11:15   2024-07-12 16:00:00
2024-07-10 13:21:10   2024-07-22 14:12:45
2024-07-11 12:01:59   2024-07-22 14:12:45
2024-07-12 16:00:00   2024-07-23 10:51:52
2024-07-16 10:07:09   2024-07-26 10:11:15
Name: time_close, Length: 1513, dtype: datetime64[ns]

In [6]:

# Step 4 : Filter events where target is higher than threshold
# Create target series
ptsl = [1,1]
target=dailyVol
# Select minRet
minRet = 0.01
events = f_ch3.getEvents(close,tEvents,ptsl,target,minRet,numThreads = 1 ,t1=t1)
events

,t1,trgt
2009-10-02 14:37:25,2009-10-07 11:39:00,0.012797
2009-10-07 11:39:00,2009-10-15 12:03:22,0.028053
2009-10-20 11:12:37,2009-10-27 11:06:15,0.020279
2009-10-27 11:06:15,2009-11-03 11:21:01,0.020823
2009-11-02 10:30:21,2009-11-10 10:55:00,0.018378
...,...,...
2023-11-02 13:19:47,2023-11-03 12:07:30,0.010664
2023-11-03 12:07:30,2023-11-14 10:00:23,0.011264
2023-11-03 15:23:46,2023-11-14 10:00:23,0.011404
2023-11-14 10:00:23,2023-11-20 13:08:22,0.011173


**(b)** Apply the function `mpNumCoEvents` to compute the number of overlapping outcomes at each point in time.

In [8]:
# **SNIPPET 4.1** **ESTIMATING THE UNIQUENESS OF A LABEL**

def mpNumCoEvents(closeIdx, t1, molecule):
    '''
    Compute the number of concurrent events per bar.
    +molecule[0] is the date of the first event on which the weight will be computed
    +molecule[-1] is the date of the last event on which the weight will be computed
    Any event that starts before t1[molecule].max() impacts the count.
    '''
    #1) find events that span the period [molecule[0], molecule[-1]]
    t1 = t1.fillna(closeIdx[-1])  # unclosed events still must impact other weights
    t1 = t1[t1 >= molecule[0]]  # events that end at or after molecule[0]
    t1 = t1.loc[:t1[molecule].max()]  # events that start at or before t1[molecule].max()
    
    #2) count events spanning a bar
    iloc = closeIdx.searchsorted(np.array([t1.index[0], t1.max()]))
    count = pd.Series(0, index=closeIdx[iloc[0]:iloc[1] + 1])
    for tIn, tOut in t1.items():  # Changed from iteritems() to items()
        count.loc[tIn:tOut] += 1
    return count.loc[molecule[0]:t1[molecule].max()]


In [12]:
close.index

DatetimeIndex(['2009-09-29 13:23:42', '2009-10-01 13:44:06',
               '2009-10-02 14:37:25', '2009-10-07 11:39:00',
               '2009-10-12 14:24:38', '2009-10-15 12:03:22',
               '2009-10-20 11:12:37', '2009-10-22 15:42:26',
               '2009-10-27 11:06:15', '2009-10-29 11:47:25',
               ...
               '2024-07-22 14:12:45', '2024-07-23 10:51:52',
               '2024-07-23 15:42:59', '2024-07-24 11:32:35',
               '2024-07-24 15:50:56', '2024-07-25 11:50:36',
               '2024-07-25 14:46:14', '2024-07-26 10:11:15',
               '2024-07-26 15:59:52', '2024-07-26 16:00:00'],
              dtype='datetime64[ns]', name='time_close', length=4231, freq=None)

In [9]:
# mpNumCoEvents(close.index, t1, events.index)
mpNumCoEvents(closeIdx = close.index, t1 = events['t1'], molecule = events.index)[-10:]

time_close
2023-11-08 12:47:57    2
2023-11-09 12:32:28    2
2023-11-10 15:55:48    2
2023-11-14 10:00:23    3
2023-11-14 15:59:59    2
2023-11-15 11:12:01    2
2023-11-16 10:23:46    2
2023-11-16 15:51:41    2
2023-11-17 16:00:00    2
2023-11-20 13:08:22    2
dtype: int64

In [10]:
mpNumCoEvents(closeIdx = close.index, t1 = t1, molecule = events.index)[-10:]

time_close
2023-11-15 11:12:01    2
2023-11-16 10:23:46    2
2023-11-16 15:51:41    2
2023-11-17 16:00:00    2
2023-11-20 13:08:22    2
2023-11-21 09:48:07    2
2023-11-21 15:26:36    2
2023-11-22 13:08:34    2
2023-11-24 10:33:05    2
2023-11-27 09:56:25    1
dtype: int64

### **(c)** Plot the time series of the number of concurrent labels on the primary axis, and the time series of exponentially weighted moving standard deviation of returns on the secondary axis.

In [ ]:
# Step 1: Compute the number of concurrent events
numCoEvents = mpNumCoEvents(close.index, t1, events.index)

# Step 2: Compute the exponentially weighted moving standard deviation of returns
SPAN = 50
ewm_std = close.pct_change().ewm(span=SPAN).std()

# Step 3: Plot the time series
fig, ax1 = plt.subplots(figsize=(14, 7))

# Plot the number of concurrent events on the primary axis
color = 'tab:blue'
ax1.set_xlabel('Date')
ax1.set_ylabel('Number of Concurrent Events', color=color)
ax1.plot(numCoEvents.index, numCoEvents, color=color, label='Concurrent Events')
ax1.tick_params(axis='y', labelcolor=color)

# Create a secondary axis to plot the exponentially weighted moving standard deviation of returns
ax2 = ax1.twinx()
color = 'tab:orange'
ax2.set_ylabel('EWM Standard Deviation of Returns', color=color)
ax2.plot(ewm_std.index, ewm_std, color=color, label='EWM Std of Returns')
ax2.tick_params(axis='y', labelcolor=color)

# Add a title and show the plot
plt.title('Number of Concurrent Events and EWM Std of Returns')
fig.tight_layout()
plt.show()

### **(d)** Produce a scatterplot of the number of concurrent labels (x-axis) and the exponentially weighted moving standard deviation of returns (y-axis). Can you appreciate a relationship?

In [ ]:
# Align the number of concurrent events and the EWM standard deviation of returns
aligned_numCoEvents = numCoEvents.reindex(ewm_std.index).dropna()
aligned_ewm_std = ewm_std.reindex(aligned_numCoEvents.index).dropna()

# Produce the scatterplot
plt.figure(figsize=(10, 6))
plt.scatter(aligned_numCoEvents, aligned_ewm_std)
plt.xlabel('Number of Concurrent Labels')
plt.ylabel('EWM Standard Deviation of Returns')
plt.title('Scatterplot: Concurrent Labels vs. EWM Std of Returns')
plt.grid(True)
plt.show()


### 4.2 Using the function `mpSampleTW`, compute the average uniqueness of each label. What is the first-order serial correlation, AR(1), of this time series? Is it statistically significant? Why?


In [ ]:
# Get the Labeled dataframe, on which we'll add the uniqueness of a label
out = f_ch3.getBinsNew(events, close, t1)
out = f_ch3.getBins(events, close)
out['bin'].value_counts()

In [64]:
# **SNIPPET 4.2** **ESTIMATING THE AVERAGE UNIQUENESS OF A LABEL**

def mpSampleTW(t1, numCoEvents, molecule):
    # Derive average uniqueness over the event's lifespan
    wght = pd.Series(index=molecule)
    for tIn, tOut in t1.loc[wght.index].items():  # Changed from .iteritems() to .items()
        wght.loc[tIn] = (1. / numCoEvents.loc[tIn:tOut]).mean()
    return wght

In [65]:
numCoEvents = f_ch3.mpPandasObj(
    mpNumCoEvents,
    ("molecule", events.index),
    numThreads=1,
    closeIdx=close.index,
    t1=events["t1"],
)
# numCoEvents

In [66]:
numCoEvents = numCoEvents.loc[~numCoEvents.index.duplicated(keep="last")]
# numCoEvents

In [67]:
numCoEvents = numCoEvents.reindex(close.index).fillna(0)
# numCoEvents

In [ ]:
out["tW"] = f_ch3.mpPandasObj(
    mpSampleTW,
    ("molecule", events.index),
    numThreads=1,
    t1=events["t1"],
    numCoEvents=numCoEvents,
)

# average uniqueness of a label
print(f"The average uniqueness of a label is: {out['tW'].mean()}")

#### First-order serial correlation

In [ ]:
# Assume `out["tW"]` contains the average uniqueness values computed earlier

# Step 1: Compute the AR(1) coefficient
ar1_coefficient = out["tW"].autocorr(lag=1)

# Step 2: Calculate the t-statistic to test for significance
n = len(out["tW"].dropna())  # Number of observations (excluding NaN values)
t_statistic = ar1_coefficient * np.sqrt((n - 2) / (1 - ar1_coefficient**2))

# Step 3: Calculate the p-value
p_value = 2 * (1 - stats.t.cdf(np.abs(t_statistic), df=n - 2))

# Output the AR(1) coefficient, t-statistic, and p-value
print(f"AR(1) Coefficient: {ar1_coefficient}")
print(f"T-statistic: {t_statistic}")
print(f"P-value: {p_value}")

# Interpretation
if p_value < 0.05:
    print("The AR(1) coefficient is statistically significant.")
else:
    print("The AR(1) coefficient is not statistically significant.")

In [ ]:
out

In [ ]:
"""
**4.3** Fit a random forest to a financial dataset where \( I^{-1} \sum_{i=1}^{I} \tilde{u}_i \ll 1 \).

- **(a)** What is the mean out-of-bag accuracy?
  
- **(b)** What is the mean accuracy of k-fold cross-validation (without shuffling) on the same dataset?
  
- **(c)** Why is out-of-bag accuracy so much higher than cross-validation accuracy? Which one is more correct / less biased? What is the source of this bias?

"""

#### **4.3** Fit a random forest to a financial dataset where $I^{-1} \sum_{i=1}^{I} \tilde{u}_i \ll 1$ .

In [ ]:
# Let's add features in the dataset to fit a random forest model
df

In [ ]:
df_ = df.copy()

# 1. Rolling Volatility (Standard Deviation)
df['rolling_vol_10'] = df['close'].rolling(window=10).std()
df['rolling_vol_20'] = df['close'].rolling(window=20).std()

# 2. Moving Averages
df['MA_10'] = df['close'].rolling(window=10).mean()
df['MA_20'] = df['close'].rolling(window=20).mean()  # Define MA_20 for Bollinger Bands
df['MA_50'] = df['close'].rolling(window=50).mean()

# 3. Exponential Moving Averages (EMA)
df['EMA_10'] = df['close'].ewm(span=10, adjust=False).mean()
df['EMA_50'] = df['close'].ewm(span=50, adjust=False).mean()

# 4. Relative Strength Index (RSI)
delta = df['close'].diff(1)
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)

avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()

rs = avg_gain / avg_loss
df['RSI'] = 100 - (100 / (1 + rs))

# 5. Bollinger Bands
df['BB_upper'] = df['MA_20'] + (df['rolling_vol_20'] * 2)
df['BB_lower'] = df['MA_20'] - (df['rolling_vol_20'] * 2)

# 6. MACD (Moving Average Convergence Divergence)
df['EMA_12'] = df['close'].ewm(span=12, adjust=False).mean()
df['EMA_26'] = df['close'].ewm(span=26, adjust=False).mean()
df['MACD'] = df['EMA_12'] - df['EMA_26']
df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

# 7. Bar duration as time_close (index) - time_open
df['bar_duration'] = df.index.to_series().diff().dt.total_seconds()

# Drop any rows with NaN values (due to rolling calculations)
df = df.dropna()

df

In [ ]:
# change index name of 'out' to 'time_close'
out.index.name = 'time_close'
out

In [ ]:
# create dataframe 'dataset' which is "out left join df on index"
dataset = pd.merge(out, df, left_index=True, right_index=True, how='inner')
dataset

In [76]:
# save 'dataset' as a pickle file "dataset_ch4_3.pkl"
path = '../../data/dataset_ch4_3.pkl'
with open(path, 'wb') as f:
    pickle.dump(dataset, f)
    

In [77]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

N_ESTIMATORS = 100

# Define the features and target
X = dataset.drop(columns=["bin", "ret", "time_open"])  # Dropping 'bin' and 'ret'
y = dataset["bin"]  # Target is the 'bin' column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, shuffle=False
)

# Create a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        (
            "num",
            Pipeline(
                [
                    # ("imputer", SimpleImputer(strategy="median")),
                    ("scaler", MinMaxScaler()),
                ]
            ),
            [
                "bar_duration",
                "close",
                "vwap",
                "rolling_vol_10",
                "rolling_vol_20",
                "MA_10",
                "MA_20",
                "MA_50",
                "EMA_10",
                "EMA_50",
                "RSI",
                "BB_upper",
                "BB_lower",
            ],
        ),
        ("macd", Pipeline([("scaler", StandardScaler())]), ["MACD"]),
    ],
    remainder="passthrough",
)

# Create a pipeline that includes both the preprocessor and the model
pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "classifier",
            RandomForestClassifier(
                n_estimators=N_ESTIMATORS,
                random_state=RANDOM_STATE,
                bootstrap=True,
                oob_score=True,
            ),
        ),
    ]
)

In [ ]:
CV_search = 20
CV = 5

# Set up GridSearchCV to optimize for accuracy and include max_features for feature bootstrapping
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__max_features': ['sqrt', 'log2', None]
}

# Initialize GridSearchCV with accuracy as the scoring metric
grid_search = GridSearchCV(pipeline, param_grid, scoring='accuracy', cv=CV, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best estimator from the grid search
best_model = grid_search.best_estimator_

# Retrieve the mean out-of-bag accuracy
oob_accuracy = best_model.named_steps['classifier'].oob_score_

# Make predictions on the test data
y_pred = best_model.predict(X_test)

# Evaluate the model
best_params = grid_search.best_params_
classification_rep = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print(f"Best Parameters: {best_params}")
print(f"OOB Accuracy: {oob_accuracy}")
print(f"Test Accuracy: {accuracy}")
print(f"Classification Report:\n{classification_rep}")

In [79]:
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import accuracy_score, classification_report
# from sklearn.ensemble import RandomForestClassifier

# # Initialize the Random Forest Classifier with bootstrapping for features and enable OOB scoring
# N_ESTIMATORS = 100
# CV = 10
# rf_clf = RandomForestClassifier(n_estimators=N_ESTIMATORS, random_state=RANDOM_STATE, bootstrap=True, oob_score=True)

# # Set up GridSearchCV to optimize for accuracy and include max_features for feature bootstrapping
# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [None, 10, 20, 30],
#     'min_samples_split': [2, 5, 10],
#     'max_features': ['auto', 'sqrt', 'log2']  # Bootstrapping features with different strategies
# }

# # Initialize GridSearchCV with accuracy as the scoring metric and enable parallelization
# grid_search = GridSearchCV(rf_clf, param_grid, scoring='accuracy', cv=CV, n_jobs=-1)  # n_jobs=-1 uses all available cores
# grid_search.fit(X_train, y_train)

# # Get the best estimator from the grid search
# rf_clf = grid_search.best_estimator_

# # Retrieve the mean out-of-bag accuracy
# oob_accuracy = rf_clf.oob_score_

# # Make predictions on the test data
# y_pred = rf_clf.predict(X_test)

# # Evaluate the model
# best_params = grid_search.best_params_
# classification_rep = classification_report(y_test, y_pred)
# accuracy = accuracy_score(y_test, y_pred)

In [ ]:
print(classification_rep)

In [ ]:
import shap
shap.initjs()

In [ ]:
# Extract the fitted RandomForestClassifier from the pipeline
rf_clf = best_model.named_steps['classifier']

# Initialize the SHAP explainer with the fitted RandomForestClassifier
explainer = shap.TreeExplainer(rf_clf)

# Calculate SHAP values for the test set
shap_values = explainer.shap_values(X_test)

# Summarize the feature importance
shap.summary_plot(shap_values[1], X_test, plot_type="bar")

In [ ]:
# Create a summary plot
shap.summary_plot(shap_values[1], X_test)

In [ ]:
# Visualize the first prediction's SHAP values
shap.force_plot(explainer.expected_value[1], shap_values[1][0], X_test.iloc[0])

In [ ]:
importances = rf_clf.feature_importances_

# Create a plot to visualize the feature importances
plt.figure(figsize=(10, 6))
plt.barh(X_train.columns, importances, color='skyblue')
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Random Forest Feature Importance')
plt.show()

### **(a)** What is the mean out-of-bag accuracy?

In [ ]:
print(f"Mean Out-of-Bag Accuracy: {oob_accuracy:.4f}")

### **(b)** What is the mean accuracy of k-fold cross-validation (without shuffling) on the same dataset?

In [ ]:
from sklearn.model_selection import cross_val_score

# Perform k-fold cross-validation with k=10 (without shuffling)
CV = 5
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=CV, scoring='accuracy')

# Calculate the mean accuracy
mean_accuracy = cv_scores.mean()
print(f"Mean Accuracy of k-fold Cross-Validation (Without Shuffling): {mean_accuracy:.4f}")



### **(c)** Why is out-of-bag accuracy so much higher than cross-validation accuracy? Which one is more correct / less biased? What is the source of this bias?

### Explanation of Metrics

#### 1. **Out-of-Bag (OOB) Accuracy**
- **Definition**: Out-of-bag (OOB) accuracy is a performance measure specific to ensemble methods like Random Forests that use bootstrap aggregating (bagging). In bagging, each tree in the forest is trained on a bootstrap sample, which is a subset of the training data. OOB accuracy is calculated using the instances that were not included in the bootstrap sample for each tree (these are referred to as the "out-of-bag" samples).
- **Purpose**: OOB accuracy provides an unbiased estimate of the model's performance without requiring a separate validation set. It's calculated during the training process, so it doesn't require any additional data splitting.
- **How It Works**: For each data point, only the trees that did not see that data point during training (because it was not in their bootstrap sample) are used to predict its label. The OOB accuracy is the proportion of correct predictions for all the out-of-bag samples.

#### 2. **K-Fold Cross-Validation Accuracy**
- **Definition**: K-fold cross-validation is a statistical method used to evaluate a model's performance by splitting the data into `k` equally sized folds. The model is trained on `k-1` folds and validated on the remaining fold. This process is repeated `k` times, with each fold being used as the validation set once. The final cross-validation accuracy is the average accuracy across all `k` iterations.
- **Purpose**: K-fold cross-validation is used to assess the generalizability of a model. It provides a more robust estimate of model performance by ensuring that every observation is used for both training and validation.
- **How It Works**: Each time the model is trained on `k-1` folds and tested on the remaining fold, this gives an estimate of how the model performs on unseen data. After repeating this process `k` times, the average accuracy is computed.

### Comparison and Answer to the Question

#### (c) Why is out-of-bag accuracy so much higher than cross-validation accuracy? Which one is more correct / less biased? What is the source of this bias?

**Observation:**
- You observed that the mean out-of-bag accuracy is higher than the mean accuracy from k-fold cross-validation.

**Reasons for the Difference:**

1. **Training Data Differences:**
   - **OOB Accuracy**: Each tree in the Random Forest is trained on a slightly different subset of the training data due to bootstrapping. The OOB accuracy is calculated on the data that was not seen by the individual trees during their training. However, since Random Forests are designed to perform well on out-of-bag samples by aggregating the predictions of multiple trees, OOB accuracy often reflects an optimistic estimate of performance.
   - **Cross-Validation Accuracy**: In contrast, during k-fold cross-validation, the model is tested on a fold of data that was completely withheld during training. This process better simulates how the model will perform on entirely unseen data, providing a more realistic estimate of the model's generalization ability.

2. **Overfitting on OOB Samples:**
   - **OOB Accuracy**: Since the OOB accuracy is calculated using only the subset of trees that did not see the data point during training, it might be slightly optimistic. Even though each tree hasn't seen the OOB samples, the ensemble as a whole has a stronger tendency to perform well on these samples due to the aggregating effect of the forest.
   - **Cross-Validation Accuracy**: K-fold cross-validation, especially when not shuffled, may capture nuances in the data that could lead to overfitting, depending on the data's structure. If the dataset has any inherent order or if the model is overfitting the training data, the cross-validation score could be lower.

3. **Bias and Correctness:**
   - **OOB Accuracy**: While OOB accuracy is useful and often close to the true performance of the model, it can be slightly optimistic due to the reasons mentioned above.
   - **Cross-Validation Accuracy**: Cross-validation accuracy is generally considered to be a more robust and less biased estimate of the model's true generalization performance, especially when done without shuffling. It provides a comprehensive evaluation by testing the model on completely unseen data during each fold.

**Conclusion:**
- **Which One Is More Correct/Less Biased?**: In most cases, k-fold cross-validation accuracy is considered more correct and less biased, particularly when you are concerned with how well the model will generalize to entirely new, unseen data. Cross-validation evaluates the model's performance across multiple splits of the data, making it a reliable measure of generalization.
- **Source of Bias**: The bias in OOB accuracy arises because the OOB samples are still part of the overall training data, even though they aren't used by the individual trees. This can lead to a slight overestimation of the model's performance.

In summary, while OOB accuracy provides a quick and useful estimate of model performance, k-fold cross-validation without shuffling generally gives a more accurate and less biased assessment of the model's generalization ability. The difference you observed, where OOB accuracy is higher than cross-validation accuracy, likely stems from the optimistic nature of OOB evaluation and the more rigorous testing applied during cross-validation.

---

In [88]:
# **SNIPPET 4.10** **DETERMINATION OF SAMPLE WEIGHT BY ABSOLUTE RETURN ATTRIBUTION**
def mpSampleW(t1, numCoEvents, close, molecule):
    # Derive sample weight by return attribution
    ret = np.log(close).diff()  # log-returns, so that they are additive
    wght = pd.Series(index=molecule)
    for tIn, tOut in t1.loc[wght.index].items():  # Changed from iteritems() to items()
        wght.loc[tIn] = (ret.loc[tIn:tOut] / numCoEvents.loc[tIn:tOut]).sum()
    return wght.abs()


In [ ]:
out["w"] = f_ch3.mpPandasObj(
    mpSampleW,
    ("molecule", events.index),
    numThreads=1,
    t1=events["t1"],
    numCoEvents=numCoEvents,
    close=close,
)

out

In [ ]:
out['w'] *= out.shape[0] / out['w'].sum()

out

---

In [ ]:
out['tW']

#### **4.4** Modify the code in Section 4.7 to apply an exponential time-decay factor

In [ ]:
# **SNIPPET 4.11** **IMPLEMENTATION OF TIME-DECAY FACTORS**


def getTimeDecay(tW, clfLastW=1.0):
    # apply piecewise-linear decay to observed uniqueness (tW)
    # newest observation gets weight=1, oldest observation gets weight=clfLastW
    clfW = tW.sort_index().cumsum()
    if clfLastW >= 0:
        slope = (1.0 - clfLastW) / clfW.iloc[-1]
    else:
        slope = 1.0 / ((clfLastW + 1) * clfW.iloc[-1])
    const = 1.0 - slope * clfW.iloc[-1]
    clfW = const + slope * clfW
    clfW[clfW < 0] = 0
    print(const, slope)
    return clfW


getTimeDecay(out["tW"], clfLastW=0.8)

In [ ]:
import numpy as np

def getExpTimeDecay(tW, clfLastW=1.0):
    # apply piecewise-linear decay to observed uniqueness (tW)
    # newest observation gets weight=1, oldest observation gets weight=clfLastW
    clfW = tW.sort_index().cumsum()
    if clfLastW > 0:
        slope = (-1.0/clfW.iloc[-1])*np.log(clfLastW)
    else:
        raise ValueError('clfLastW must be greater than 0')
    const = - slope * clfW.iloc[-1]
    clfW = np.exp(const + slope * clfW)
    clfW[clfW < 0] = 0
    print(const, slope)
    return clfW


getExpTimeDecay(out["tW"], clfLastW=0.8)

In [ ]:
out['tW']